Import Library

In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

In [2]:
# from google.colab import drive
# drive.mount('/gdrive')
# %cd /gdrive

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

Load Dataset

In [4]:
dataset = pd.read_csv('NASDAQCompositeDataset.csv', index_col=0)
#dataset = dataset.iloc[::-1] # reverse the order of dataset


Dataset Preview

In [5]:
dataset

,Date,Price,Open,High,Low,Vol.,Change %
No,,,,,,,
0,9/6/2017,"6,393.31","6,394.35","6,407.40","6,356.20",419.28M,0.28%
1,9/7/2017,"6,397.87","6,402.94","6,413.07","6,379.81",433.05M,0.07%
2,9/8/2017,"6,360.19","6,389.65","6,391.41","6,354.96",408.11M,-0.59%
3,9/11/2017,"6,432.26","6,411.18","6,439.11","6,410.71",431.57M,1.13%
4,9/12/2017,"6,454.28","6,448.81","6,455.02","6,429.54",404.69M,0.34%
...,...,...,...,...,...,...,...
1372,2/17/2023,"11,787.27","11,777.50","11,803.22","11,673.21",966.30M,-0.58%
1373,2/21/2023,"11,492.30","11,640.37","11,684.14","11,491.18",1.02B,-2.50%
1374,2/22/2023,"11,507.07","11,517.20","11,582.52","11,445.17",942.98M,0.13%


Preproccessing Data

1. Filtering Data

In [6]:
dataset=dataset.drop(['Price', 'High', 'Low', 'Vol.', "Change %"], axis=1)
dataset["Date"] = pd.to_datetime(dataset['Date'])
dataset['Open']= dataset['Open'].str.replace(',', '')
dataset

,Date,Open
No,,
0,2017-09-06,6394.35
1,2017-09-07,6402.94
2,2017-09-08,6389.65
3,2017-09-11,6411.18
4,2017-09-12,6448.81
...,...,...
1372,2023-02-17,11777.50
1373,2023-02-21,11640.37
1374,2023-02-22,11517.20


2. Filling Missing Value

3. Reshape Data

4. Normalization Data

5. Split Data

6. Create Data Train

7. Ekstraksi Fitur dengan CNN

8. Flattening Feature Map

Split Dataset into 3 range time different

In [7]:
data_before_pandemic=dataset[dataset["Date"] < "3/11/2020"]
data_while_pandemic=dataset[(dataset["Date"] >= "3/11/2020") & (dataset["Date"] <= "9/13/2022")]
data_after_pandemic=dataset[dataset["Date"] > "9/13/2022"]

#Dataset before pandemic

In [8]:
data_before_pandemic

,Date,Open
No,,
0,2017-09-06,6394.35
1,2017-09-07,6402.94
2,2017-09-08,6389.65
3,2017-09-11,6411.18
4,2017-09-12,6448.81
...,...,...
626,2020-03-04,8834.10
627,2020-03-05,8790.09
628,2020-03-06,8469.01


#Dataset while pandemic

In [9]:
data_while_pandemic

,Date,Open
No,,
631,2020-03-11,8136.25
632,2020-03-12,7398.58
633,2020-03-13,7610.39
634,2020-03-16,7392.73
635,2020-03-17,6904.59
...,...,...
1259,2022-09-07,11559.38
1260,2022-09-08,11679.86
1261,2022-09-09,11958.61


#Dataset after pandemic

In [10]:
data_after_pandemic

,Date,Open
No,,
1264,2022-09-14,11680.41
1265,2022-09-15,11633.24
1266,2022-09-16,11401.21
1267,2022-09-19,11338.57
1268,2022-09-20,11440.14
...,...,...
1372,2023-02-17,11777.50
1373,2023-02-21,11640.37
1374,2023-02-22,11517.20


Normalize Dataset

In [11]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
dataset['Date'] = pd.to_numeric(pd.to_datetime(dataset['Date']))
dataset["Open"] = pd.to_numeric(dataset["Open"])
dataset= scaler.fit_transform(dataset)


Split Dataset Into Train and Test Sets

In [12]:
train_size = int(len(dataset) * 0.8)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

Define a function to create a dataset with look back

In [13]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

Reshape dataset into X=t and Y=t+1

In [14]:
look_back = 3
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

Reshape input to be [samples, time steps, features]

In [15]:
trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 1))

Create and Fit the CNN-LSTM model

In [16]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(look_back, 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=64, verbose=2)

Epoch 1/100
18/18 - 6s - loss: 0.1147 - 6s/epoch - 354ms/step
Epoch 2/100
18/18 - 0s - loss: 0.0116 - 90ms/epoch - 5ms/step
Epoch 3/100
18/18 - 0s - loss: 0.0059 - 102ms/epoch - 6ms/step
Epoch 4/100
18/18 - 0s - loss: 0.0031 - 122ms/epoch - 7ms/step
Epoch 5/100
18/18 - 0s - loss: 0.0024 - 99ms/epoch - 5ms/step
Epoch 6/100
18/18 - 0s - loss: 0.0022 - 106ms/epoch - 6ms/step
Epoch 7/100
18/18 - 0s - loss: 0.0019 - 68ms/epoch - 4ms/step
Epoch 8/100
18/18 - 0s - loss: 0.0017 - 70ms/epoch - 4ms/step
Epoch 9/100
18/18 - 0s - loss: 0.0017 - 69ms/epoch - 4ms/step
Epoch 10/100
18/18 - 0s - loss: 0.0016 - 79ms/epoch - 4ms/step
Epoch 11/100
18/18 - 0s - loss: 0.0016 - 112ms/epoch - 6ms/step
Epoch 12/100
18/18 - 0s - loss: 0.0016 - 103ms/epoch - 6ms/step
Epoch 13/100
18/18 - 0s - loss: 0.0016 - 103ms/epoch - 6ms/step
Epoch 14/100
18/18 - 0s - loss: 0.0014 - 123ms/epoch - 7ms/step
Epoch 15/100
18/18 - 0s - loss: 0.0015 - 117ms/epoch - 6ms/step
Epoch 16/100
18/18 - 0s - loss: 0.0014 - 90ms/epoch - 5m

Make predictions

In [17]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
trainPredictPlot = np.empty_like(dataset)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
testPredictPlot = np.empty_like(dataset)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict


Invert predictions back to original scale

In [18]:
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])


ValueError: non-broadcastable output operand with shape (1097,1) doesn't match the broadcast shape (1097,2)

calculate root mean squared error (RMSE)

In [ ]:
from sklearn.metrics import mean_squared_error
trainScore = np.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train RMSE: %.2f' % (trainScore))
testScore = np.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test RMSE: %.2f' % (testScore))